<a href="https://colab.research.google.com/github/BonfaDScience/BonfaDScience/blob/main/reducao_dimensionalidade_imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Leitura e manipulação da imagem PPM
from PIL import Image # Importa a biblioteca PIL para processamento de imagens

def read_ppm(filename):
    """Lê uma imagem no formato PPM (P6) ou no formato jpeg e retorna a largura, a altura e as info pixel."""
    try:
        # Tenta abrir um arquivo PPM primeiro
        with open(filename, 'rb') as f:
            header = f.readline().decode()  # Verifica se é um P6
            if header.strip() != 'P6':
                raise ValueError("Não é um arquivo PPM P6")

            # Leitura dos metadados da imagem (largura, altura, valor máximo da cor)
            while True:
                line = f.readline().decode()
                if line.startswith('#'):  # Ignora os comentários
                    continue
                width, height = map(int, line.split())
                break

            max_val = int(f.readline().decode().strip())  # Lê o valor máximo das cores (p.e., 255)
            if max_val != 255:
                raise ValueError("Apenas o valor máximo de 255 é válido.")

            # Lê as info dos pixels (binário)
            pixel_data = f.read()

        return width, height, pixel_data
    except UnicodeDecodeError:
        # Se a leitura do PPM falhar, tentará abrir como jpeg usando o PIL
        img = Image.open(filename)
        # Converte para RGB se necessário
        img = img.convert("RGB")
        # Determina as dimensões da imagem
        width, height = img.size
        # Determina as info pixel em bytes
        pixel_data = img.tobytes()
        return width, height, pixel_data

In [19]:
# Conversão RGB para tons de cinza

def rgb_to_grayscale(pixel_data):
    """Converte as informações RGB dos pixels em escala de cinza usando a fórmula de luminância (quantifica o brilho de uma cor ou de um pixel para determinar o nível de cinza de cada pixel)."""
    grayscale_data = bytearray()
    for i in range(0, len(pixel_data), 3):
        r, g, b = pixel_data[i], pixel_data[i+1], pixel_data[i+2]
        # Fórmula da luminância para conversão em escala de cinza
        gray = int(0.299 * r + 0.587 * g + 0.114 * b)
        grayscale_data.extend([gray, gray, gray])
    return grayscale_data

In [20]:
# Conversão de cinza para preto e branco

def grayscale_to_binary(grayscale_data, threshold=128):
    """Converte a imagem em escala de cinza em binário (preto e branco) usando um corte(128)."""
    binary_data = bytearray()
    for i in range(0, len(grayscale_data), 3):
        gray = grayscale_data[i]  # Pega qualquer canal, desde que em escala de cinza
        value = 255 if gray > threshold else 0  # Binarização ocorre aqui
        binary_data.extend([value, value, value])
    return binary_data

In [21]:
# Salvando imagem em formato PPM

def save_ppm(filename, width, height, pixel_data):
    """Salva as infos pixel num arquivo PPM (P6)."""
    with open(filename, 'wb') as f:
        f.write(f'P6\n{width} {height}\n255\n'.encode())
        f.write(pixel_data)

In [29]:
# Salvando imagens processadas em PPM e PNG

import requests
from PIL import Image
from io import BytesIO
from google.colab import files

def download_image(url):
    """Download da imagem à partir de uma URL retorna largura, altura e info pixel."""
    response = requests.get(url)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        img = img.convert("RGB")  # Garantir que está em RGB
        width, height = img.size
        pixel_data = img.tobytes()  # Converter para bytes
        return width, height, pixel_data
    else:
        raise ValueError("Falha ao fazer o download da imagem")

def save_as_ppm(filename, width, height, pixel_data):
    """Salva as infos do pixel em uma imagem PPM (P6)."""
    with open(filename, 'wb') as f:
        f.write(f'P6\n{width} {height}\n255\n'.encode())  # PPM header
        f.write(bytes(pixel_data))  # Escreve as informações pixel no formato binário

def save_as_png(filename, width, height, pixel_data):
    """Salva as infos RGB do pixel no formato .png."""
    img = Image.frombytes('RGB', (width, height), bytes(pixel_data))
    img.save(filename, format='PNG')

# #Caso queira fazer upload da imagem (PPM ou JPEG) à partir da própria máquina do usuário
# uploaded = files.upload()
# input_image = list(uploaded.keys())[0]
# width, height, pixel_data = read_ppm(input_image)

# URL da imagem
#fiz testes lendo a imagem diretamente do seu site
#image_url = "https://thumbs.dreamstime.com/b/morena-baccarin-23574798.jpg"
image_url = "https://m.media-amazon.com/images/M/MV5BMDliNjQxZmQtZDM2ZS00MzEwLTliMjQtYTlmYTdiYTI3OWIyXkEyXkFqcGc@._V1_FMjpg_UX1000_.jpg"
#image_url = "https://external-preview.redd.it/N2mrfYNmoB0THixU4kU9V9sSkb60_rCWULeBFqvdaH4.jpg?width=640&crop=smart&auto=webp&s=7dd3f96f485cad603f4a96793a521ebcded7466c"

# Baixando a imagem da URL
width, height, pixel_data = download_image(image_url)

# Processamento da imagem
grayscale_data = rgb_to_grayscale(pixel_data)
binary_data = grayscale_to_binary(grayscale_data, threshold=128)

# Salvando como PPM e PNG
save_as_ppm('baccarin_grayscale.ppm', width, height, grayscale_data)
save_as_png('baccarin_grayscale.png', width, height, grayscale_data)
save_as_ppm('baccarin_binary.ppm', width, height, binary_data)
save_as_png('baccarin_binary.png', width, height, binary_data)

print("Imagem original convertida para escala de cinza e binário.")

# Download dos arquivos gerados
for file_name in ['baccarin_grayscale.png', 'baccarin_binary.png']:
    files.download(file_name)
    print(f"Download do arquivo: {file_name}")

print("O download das imagens foi completado com sucesso.")





Imagem original convertida para escala de cinza e binário.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download do arquivo: baccarin_grayscale.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download do arquivo: baccarin_binary.png
O download das imagens foi completado com sucesso.
